In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os 
os.chdir("/content/drive/My Drive/Group_Project")


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from __future__ import print_function
import PIL.Image as Image
import matplotlib.pyplot as plt
import copy
import numpy as np

use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

The following code for the NST algorithm was referenced from the work of 
Sunghyun (psh01087) on GitHub.

In [ ]:
imsize = 256 if use_cuda else 128

transform = transforms.Compose([transforms.Resize([imsize,imsize]), transforms.ToTensor()])

def image_loader(image_name, transform=None, max_size=None, shape=None):
    image = Image.open(image_name).convert('RGB')
    
    if max_size is not None:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.ANTIALIAS)
    
    if shape is not None:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform is not None:
        image = transform(image)
        
    image = Variable(image)
    image = image.unsqueeze(0)
    
    return image.type(dtype)

In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion(input * self.weight, self.target)
        self.output = input
        return self.output

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

In [ ]:
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()
        features = input.view(a * b, c * d)
        G = torch.mm(features, features.t())
        return G.div(a * b * c * d)

In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target, weight):
        super(StyleLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.gram = GramMatrix()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.output = input.clone()
        self.G = self.gram(input)
        self.G.mul_(self.weight)
        self.loss = self.criterion(self.G, self.target)
        return self.output

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

In [ ]:
cnn = models.vgg19(pretrained=True).features

# move it to the GPU if possible:
if use_cuda:
    cnn = cnn.cuda()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']


def get_style_model_and_losses(cnn, style_img, content_img,
                               style_weight=2000, content_weight=1,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    model = nn.Sequential()  # the new Sequential module network
    gram = GramMatrix()  # we need a gram module in order to compute style targets

    # move these modules to the GPU if possible:
    if use_cuda:
        model = model.cuda()
        gram = gram.cuda()

    i = 1
    for layer in list(cnn):
        if isinstance(layer, nn.Conv2d):
            name = "conv_" + str(i)
            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).clone()
                content_loss = ContentLoss(target, content_weight)
                model.add_module("content_loss_" + str(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).clone()
                target_feature_gram = gram(target_feature)
                style_loss = StyleLoss(target_feature_gram, style_weight)
                model.add_module("style_loss_" + str(i), style_loss)
                style_losses.append(style_loss)

        if isinstance(layer, nn.ReLU):
            name = "relu_" + str(i)
            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).clone()
                content_loss = ContentLoss(target, content_weight)
                model.add_module("content_loss_" + str(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).clone()
                target_feature_gram = gram(target_feature)
                style_loss = StyleLoss(target_feature_gram, style_weight)
                model.add_module("style_loss_" + str(i), style_loss)
                style_losses.append(style_loss)

            i += 1

        if isinstance(layer, nn.MaxPool2d):
            name = "pool_" + str(i)
            model.add_module(name, layer)  # ***

    return model, style_losses, content_losses

In [ ]:
# Optimizer (LBFGS)
def get_input_param_optimizer(input_img):
    input_param = nn.Parameter(input_img.data)
    optimizer = optim.LBFGS([input_param])
    return input_param, optimizer

In [ ]:
import time

In [ ]:
def train_style_transfer(cnn, content_img, style_img, input_img, num_steps=300,
                       style_weight=3500, content_weight=1):
    start_time = time.time()
    print('Starting style transfer!!')
    model, style_losses, content_losses = get_style_model_and_losses(cnn, style_img, content_img, style_weight, content_weight)
    input_param, optimizer = get_input_param_optimizer(input_img)

    ss = []
    cs = []
    
    run = [0]
    while run[0] <= num_steps:
        def steps():
            input_param.data.clamp_(0, 1)
            optimizer.zero_grad()
            model(input_param)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.backward()
            for cl in content_losses:
                content_score += cl.backward()

            run[0] += 1
            ss.append(style_score.item())
            cs.append(content_score.item())
            if run[0] % 50 == 0:
                print("run {}: Style Loss : {:4f} Content Loss: {:4f}".format(run, style_score.item(), content_score.item()))

            return style_score + content_score

        optimizer.step(steps)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(elapsed_time)
    # a last correction...
    input_param.data.clamp_(0, 1)

    return input_param.data, ss, cs

#training loop

**face_sketches_path**: change to path of folder containing the content images

**style_image_path**: change to path of folder containing the style images. 

In [ ]:
face_sketches_path = "/content/drive/My Drive/Group_Project/1194GoodImages"
style_images_path = "/content/drive/My Drive/Group_Project/1194AbstractImages"

In [ ]:
sketches_list = []
style_list = []

In [ ]:
for sketch_image in os.listdir(face_sketches_path):
  #if sketch_image.endswith(".jpg"):
  sketches_list.append(sketch_image)
  #else:
    #continue

In [ ]:
for style_image in os.listdir(style_images_path):
  #if style_image.endswith(".jpg"):
  style_list.append(style_image)
  #else:
    #continue

In [ ]:
len(style_list)

1194

In [ ]:
out_list = []
for i in range(1194):
  out_list.append(str(i))

In [ ]:
out2_list = []
for i in range(1194):
  out2_list.append(str(i) + '-2')

**imsize** defines the output dimesion (height x width). 

In [ ]:
imsize = 256 if use_cuda else 128
transform = transforms.Compose([transforms.Resize([imsize,imsize]), transforms.ToTensor()])

In [ ]:
for i in range(878, len(sketches_list)):
  try: 
    content_image_path = "/content/drive/My Drive/Group_Project/1194GoodImages/"
    content_image_path = content_image_path + sketches_list[i]
    style_image_path = "/content/drive/My Drive/Group_Project/1194AbstractImages/"
    style_image_path = style_image_path + style_list[i]

    content_img = image_loader(content_image_path, transform, max_size=imsize)
    style_img = image_loader(style_image_path, transform, shape=[content_img.size(2), content_img.size(3)])
    input_img = content_img.clone()

    cnn = models.vgg19(pretrained=True).features
    if use_cuda:
      cnn = cnn.cuda()

    output,ss ,cs = train_style_transfer(cnn, content_img, style_img, input_img)
    nout = output.squeeze()
    nout = nout.permute(1,2,0)

    output_path = "/content/drive/My Drive/Group_Project/output_good/" + out2_list[i] + '.jpeg'
    torchvision.utils.save_image(nout.permute(2,0,1), output_path)
  except:
    pass

Starting style transfer!!
run [50]: Style Loss : 69.868942 Content Loss: 5.167918
run [100]: Style Loss : 30.810429 Content Loss: 5.540665
run [150]: Style Loss : 14.101691 Content Loss: 5.543618
run [200]: Style Loss : 5.667924 Content Loss: 5.589829
run [250]: Style Loss : 3.136970 Content Loss: 5.518428
run [300]: Style Loss : 2.208555 Content Loss: 5.318726
75.50424885749817
Starting style transfer!!
run [50]: Style Loss : 96.535545 Content Loss: 2.989585
run [100]: Style Loss : 12.593591 Content Loss: 3.473477
run [150]: Style Loss : 3.416120 Content Loss: 3.475327
run [200]: Style Loss : 2.046199 Content Loss: 3.330994
run [250]: Style Loss : 1.467724 Content Loss: 3.213341
run [300]: Style Loss : 1.144699 Content Loss: 3.131729
75.20150804519653
Starting style transfer!!
run [50]: Style Loss : 79.435852 Content Loss: 8.428481
run [100]: Style Loss : 24.785551 Content Loss: 8.656047
run [150]: Style Loss : 12.543562 Content Loss: 8.723757
run [200]: Style Loss : 6.742910 Content 

In [ ]:
for i in range(len(sketches_list)):
  try: 
    content_image_path = "/content/drive/My Drive/Group_Project/1194GoodImages/"
    content_image_path = content_image_path + sketches_list[i]
    style_image_path = "/content/drive/My Drive/Group_Project/1194AbstractImages/"
    style_image_path = style_image_path + style_list[i]

    content_img = image_loader(content_image_path, transform, max_size=imsize)
    style_img = image_loader(style_image_path, transform, shape=[content_img.size(2), content_img.size(3)])
    input_img = content_img.clone()

    cnn = models.vgg19(pretrained=True).features
    if use_cuda:
      cnn = cnn.cuda()

    output,ss ,cs = train_style_transfer(cnn, content_img, style_img, input_img)
    nout = output.squeeze()
    nout = nout.permute(1,2,0)

    output_path = "/content/drive/My Drive/Group_Project/output_good/" + out2_list[i] + '.jpeg'
    torchvision.utils.save_image(nout.permute(2,0,1), output_path)
  except:
    pass

In [ ]:
cnn = models.vgg19(pretrained=True).features

# move it to the GPU if possible:
if use_cuda:
    cnn = cnn.cuda()

In [ ]:
"/content/drive/My Drive/Group_Project/output_good/" + out_list[0] + '.jpeg'